# Проверка качества данных: Источник дефолтов

**Таблица:** `sandbox_ai.tmp_defaults_svy`  
**Формат источника:** pkl (pickle)  
**Содержание:** история дефолтов клиентов банка (2009–2025)

## 0. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# === НАСТРОЙКИ ===
FILE_PATH = "../sources/data_defaults.pkl"

KNOWN_DEFAULT_REASONS = {
    "default_90", "def_reserve", "def_bankrupt", "def_restructure",
    "def_cross", "def_judgement", "def_sign_bankrupt",
}
DATE_FORMAT = "%d.%m.%Y"
DATE_MIN = datetime(2009, 1, 1)
DATE_MAX = datetime(2025, 12, 31)

In [ ]:
# Загрузка pkl
df = pd.read_pickle(FILE_PATH)
# Приводим все колонки к строкам для единообразных проверок
df = df.astype(str).replace("nan", np.nan)
df.columns = df.columns.str.strip()

print(f"Загружено строк: {len(df):,}, колонок: {len(df.columns)}")
print(f"Колонки: {list(df.columns)}")
df.head()

## 1. Пропуски (NULL / пустые значения)

In [ ]:
def is_empty(s):
    return s.isna() | (s.astype(str).str.strip() == "")

nulls = df.apply(lambda c: is_empty(c).sum())
pct = (nulls / len(df) * 100).round(1)
null_report = pd.DataFrame({"Пропусков": nulls, "%": pct})
null_report["Статус"] = null_report["Пропусков"].apply(lambda x: "[!]" if x > 0 else "OK")
display(null_report)

# Обязательные поля
for col in ["inn", "default_reason", "start_date"]:
    if col in df.columns and nulls.get(col, 0) > 0:
        print(f"\n!!! КРИТИЧНО: '{col}' — {nulls[col]} пропусков!")

# cure_date и finish_date могут быть пустыми
for col in ["cure_date", "finish_date"]:
    if col in df.columns:
        cnt = is_empty(df[col]).sum()
        print(f"\n[i] {col} пуст у {cnt} записей ({cnt/len(df)*100:.1f}%) — допустимо (клиент ещё в дефолте)")

## 2. Дубликаты

In [ ]:
# Полные дубликаты
print(f"Полные дубликаты строк: {df.duplicated().sum()}")

# Повторяющиеся ИНН (у клиента может быть несколько дефолтов)
if "inn" in df.columns:
    inn_counts = df["inn"].value_counts()
    print(f"Уникальных ИНН: {df['inn'].nunique():,}")
    print(f"ИНН с >1 записью: {(inn_counts > 1).sum():,}")
    if (inn_counts > 1).any():
        print("\nРаспределение кол-ва дефолтов на клиента:")
        display(inn_counts.value_counts().sort_index().rename_axis("Дефолтов").reset_index(name="Клиентов"))

# Дубликаты по паре inn + start_date
if "inn" in df.columns and "start_date" in df.columns:
    pair_dupes = df.duplicated(subset=["inn", "start_date"]).sum()
    print(f"\nДубликаты inn + start_date: {pair_dupes}")

## 3. Формат и валидность значений

In [ ]:
# --- ИНН ---
if "inn" in df.columns:
    inn_col = df["inn"].dropna().astype(str).str.strip()
    inn_col = inn_col[inn_col != ""]
    pat_digits = r'^\d+$'
    bad_chars = (~inn_col.str.match(pat_digits)).sum()
    bad_len = (~inn_col.str.len().isin([10, 12])).sum()
    len_dist = dict(inn_col.str.len().value_counts().sort_index())
    print("ИНН:")
    print(f"  Не только цифры: {bad_chars}")
    print(f"  Длина не 10 и не 12: {bad_len}")
    print(f"  Распределение по длине: {len_dist}")

In [ ]:
# --- Проверка всех дат ---
for dcol in ["start_date", "cure_date", "finish_date"]:
    if dcol not in df.columns:
        continue
    raw = df[dcol].astype(str).str.strip()
    filled = raw[(raw != "") & (raw != "nan") & (raw != "None")]
    if len(filled) == 0:
        print(f"{dcol}: все пусто")
        continue
    parsed = pd.to_datetime(filled, format=DATE_FORMAT, errors="coerce")
    bad_fmt = parsed.isna().sum()
    valid = parsed.dropna()
    before = (valid < DATE_MIN).sum()
    after = (valid > DATE_MAX).sum()
    print(f"{dcol}: заполнено {len(filled):,}, некорр. формат {bad_fmt}, "
          f"ранее {DATE_MIN.year}: {before}, позднее {DATE_MAX.year}: {after}")
    if len(valid) > 0:
        print(f"  Диапазон: {valid.min().strftime(DATE_FORMAT)} — {valid.max().strftime(DATE_FORMAT)}")

In [ ]:
# --- writeoff и unlimited_default: ожидаем 0/1 ---
for col in ["writeoff", "unlimited_default"]:
    if col not in df.columns:
        continue
    vals = set(df[col].dropna().astype(str).str.strip().unique()) - {""}
    unexpected = vals - {"0", "1"}
    print(f"{col}: уникальные = {sorted(vals)}")
    if unexpected:
        print(f"  [!] Неожиданные (ожидали 0/1): {sorted(unexpected)}")
    else:
        print(f"  [OK]")

# --- sequence_of_defaults: числовые последовательности ---
if "sequence_of_defaults" in df.columns:
    seq_col = df["sequence_of_defaults"].dropna().astype(str).str.strip()
    seq_col = seq_col[seq_col != ""]
    bad = seq_col.apply(lambda x: not all(p.strip().isdigit() for p in x.split(";"))).sum()
    print(f"\nsequence_of_defaults — нечисловые элементы: {bad}")

## 4. Справочные значения

In [ ]:
for col in ["default_reason", "reasons_on_last_date_month"]:
    if col not in df.columns:
        continue
    vals = set(df[col].dropna().astype(str).str.strip().unique()) - {""}
    unknown = vals - KNOWN_DEFAULT_REASONS
    print(f"{col}: {sorted(vals)}")
    if unknown:
        print(f"  [?] Неизвестные: {sorted(unknown)}")
    else:
        print(f"  [OK]")
    print()

if "default_reason" in df.columns:
    print("Распределение по причинам дефолта:")
    display(df["default_reason"].value_counts().rename_axis("Причина").reset_index(name="Кол-во"))

## 5. Логическая согласованность

In [ ]:
def pdate(series):
    return pd.to_datetime(series, format=DATE_FORMAT, errors="coerce")

# --- cure_date >= start_date ---
if "start_date" in df.columns and "cure_date" in df.columns:
    sd, cd = pdate(df["start_date"]), pdate(df["cure_date"])
    bad = sd.notna() & cd.notna() & (cd < sd)
    print(f"cure_date ранее start_date: {bad.sum()}")
    if bad.sum() > 0:
        display(df[bad][["inn", "start_date", "cure_date"]].head())

# --- finish_date >= cure_date ---
if "cure_date" in df.columns and "finish_date" in df.columns:
    cd, fd = pdate(df["cure_date"]), pdate(df["finish_date"])
    bad = cd.notna() & fd.notna() & (fd < cd)
    print(f"finish_date ранее cure_date: {bad.sum()}")

# --- unlimited_default=1 -> cure_date и finish_date пусты ---
if "unlimited_default" in df.columns:
    ud = df["unlimited_default"].astype(str).str.strip() == "1"
    if "cure_date" in df.columns:
        filled = ud & ~is_empty(df["cure_date"])
        print(f"\nunlimited_default=1, но cure_date заполнена: {filled.sum()}")
    if "finish_date" in df.columns:
        filled = ud & ~is_empty(df["finish_date"])
        print(f"unlimited_default=1, но finish_date заполнена: {filled.sum()}")

# --- unlimited_default=0 -> cure_date заполнена ---
if "unlimited_default" in df.columns and "cure_date" in df.columns:
    not_ud = df["unlimited_default"].astype(str).str.strip() == "0"
    empty_cure = not_ud & is_empty(df["cure_date"])
    print(f"unlimited_default=0, но cure_date пуста: {empty_cure.sum()}")

In [ ]:
# --- Длина sequence_of_defaults = длина sequence_of_dates ---
if "sequence_of_defaults" in df.columns and "sequence_of_dates" in df.columns:
    sd_col = df["sequence_of_defaults"].astype(str).str.strip()
    dt_col = df["sequence_of_dates"].astype(str).str.strip()
    mask = (sd_col != "") & (sd_col != "nan") & (dt_col != "") & (dt_col != "nan")
    len_sd = sd_col[mask].str.split(";").str.len()
    len_dt = dt_col[mask].str.split(";").str.len()
    mismatch = (len_sd != len_dt).sum()
    print(f"Несовпадение длины sequence_of_defaults и sequence_of_dates: {mismatch}")

# --- writeoff=1 при unlimited_default=0 ---
if "writeoff" in df.columns and "unlimited_default" in df.columns:
    wo_not_ud = (
        (df["writeoff"].astype(str).str.strip() == "1") &
        (df["unlimited_default"].astype(str).str.strip() == "0")
    ).sum()
    print(f"writeoff=1 при unlimited_default=0 (нетипично): {wo_not_ud}")

## 6. Статистический профиль

In [ ]:
# По годам начала дефолта
if "start_date" in df.columns:
    dates = pd.to_datetime(df["start_date"], format=DATE_FORMAT, errors="coerce")
    print("Распределение по годам начала дефолта:")
    display(dates.dt.year.value_counts().sort_index().rename_axis("Год").reset_index(name="Кол-во"))

# writeoff / unlimited_default
for col, label in [("writeoff", "Списание"), ("unlimited_default", "Бессрочный дефолт")]:
    if col in df.columns:
        print(f"\n{label} ({col}):")
        display(df[col].value_counts().rename_axis("Значение").reset_index(name="Кол-во"))

In [ ]:
# Длительность дефолтов
if "start_date" in df.columns and "cure_date" in df.columns:
    sd = pd.to_datetime(df["start_date"], format=DATE_FORMAT, errors="coerce")
    cd = pd.to_datetime(df["cure_date"], format=DATE_FORMAT, errors="coerce")
    dur = (cd - sd).dt.days
    dur = dur[dur >= 0].dropna()
    if len(dur) > 0:
        print(f"Длительность дефолтов (start -> cure), записей: {len(dur):,}")
        print(f"  Средняя: {dur.mean():.0f} дней")
        print(f"  Медиана: {dur.median():.0f} дней")
        print(f"  Мин: {dur.min():.0f}, Макс: {dur.max():.0f}")

# Длина последовательностей
if "sequence_of_defaults" in df.columns:
    seq = df["sequence_of_defaults"].dropna().astype(str).str.strip()
    seq = seq[seq != ""]
    seq_lens = seq.str.split(";").str.len()
    print(f"\nДлина последовательностей дефолтов:")
    display(seq_lens.value_counts().sort_index().rename_axis("Шагов").reset_index(name="Записей"))